In [3]:
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns

In [4]:
from k_means_constrained import KMeansConstrained
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import rpy2
import rpy2.robjects as ro
import os
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
from rpy2.robjects.packages import importr
import time
import random
import time
from numpy.random import seed
from scipy.stats import f
from PIL import Image


In [5]:
df_ = pd.read_csv("https://raw.githubusercontent.com/thistleknot/Python-Stock/master/data/raw/states.csv").set_index('States')

In [6]:
df = sm.add_constant(df_)

In [7]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}


In [8]:
def f_test(x, y, alt="two_sided"):
    """
    Calculates the F-test.
    :param x: The first group of data
    :param y: The second group of data
    :param alt: The alternative hypothesis, one of "two_sided" (default), "greater" or "less"
    :return: a tuple with the F statistic value and the p-value.
    """
    df1 = len(x) - 1
    df2 = len(y) - 1
    f = x.var() / y.var()
    if alt == "greater":
        p = 1.0 - st.f.cdf(f, df1, df2)
    elif alt == "less":
        p = st.f.cdf(f, df1, df2)
    else:
        # two-sided by default
        # Crawley, the R book, p.355
        p = 2.0*(1.0 - st.f.cdf(f, df1, df2))
    return f, p

In [9]:
wd = os.getcwd()

if (os.defpath==".;C:\\bin"):
    os.environ['R_HOME'] = 'C:/Users/User/Documents/R/R-4.1.2'
    os.environ['R_LIBS'] = 'C:/Users/User/Documents/R/R-4.1.2/library'
    from OLS_LR_DiagnosticPlots.ModelDiagnostics import Plot
else:
    os.environ['R_HOME'] = '/mnt/distvol/R/4.0.5/lib64/R/'

pandas2ri.activate()

base = importr('base')
grdevices = importr('grDevices')
print(base._libPaths())

#urbnmapr = importr('urbnmapr')
dplyr = importr('dplyr')
tidyverse = importr('tidyverse')


[1] "C:/Users/User/Documents/R/R-4.1.2/library"



In [10]:
ro.r('''

f <- function(df) {
library(urbnmapr)
library(viridis)
library(hrbrthemes)
library(dplyr)
#print(df)
spatial_data <- left_join(get_urbn_map(map = "states", sf = TRUE),
                          statedata,
                          by = "state_name")

spatial_data <- spatial_data[spatial_data$state_name %in% df$States,] %>% left_join(df, by = c('state_name' = 'States'))
spatial_data$cluster <- factor(spatial_data$cluster)

print(paste("C:\\\\Users\\\\User\\\\Documents\\\\wiki\\\\wiki\\\\dev\\\\R\\\\","clusterMap",sep = "",".png"),width = 1200, height = 800, units = "px", pointsize = 12, quality = 90)
jpeg(paste("C:\\\\Users\\\\User\\\\Documents\\\\wiki\\\\wiki\\\\dev\\\\R\\\\","clusterMap",sep = "",".png"),width = 1200, height = 800, units = "px", pointsize = 12, quality = 90)

p <- ggplot() +
  geom_sf(spatial_data,
          mapping = aes(fill = cluster),
          color = "#ffffff", size = 0.25) + scale_fill_discrete() +
  geom_sf_text(data = get_urbn_labels(map = "states", sf = TRUE), 
               aes(label = state_abbv), 
               size = 3)
  labs(fill = "cluster")

plot(p)
dev.off() 

}

''')

r_f = ro.globalenv['f']

In [30]:

#nested functions (kind of like classes, get access to parent functions members).
def filter_df(x,y):
    
    def print_df(aboveCenter,belowCenter,center):
        
        data = df[[y,*x]].sort_values(kind="quicksort", by=y, ascending=True)
        #abovePos = [i for i, item in enumerate(data.columns) if item in aboveMedian]
        #belowPos = [i for i, item in enumerate(data.columns) if item in belowMedian]

        print("all group")
        print(data.describe())
        med = pd.DataFrame(np.median(data,axis=0)).T
        means = pd.DataFrame(np.mean(data,axis=0)).T
        
        med.columns = data.columns
        
        if(center):
            centers = means
            
        else:
            centers = med
            
        filtered = data.copy()

        filtered = filtered.iloc[pd.DataFrame(np.array(filtered[list(aboveCenter)])>np.array(centers[list(aboveCenter)])).replace(False,np.NaN).dropna().index]

        filtered = filtered.iloc[pd.DataFrame(np.array(filtered[list(belowCenter)])<np.array(centers[list(belowCenter)])).replace(False,np.NaN).dropna().index]

        #opposite = ([i for i, item in enumerate(data.columns) if item not in [filtered.index]])
        oppositefilter = data.copy()

        oppositefilter = oppositefilter.iloc[pd.DataFrame(np.array(oppositefilter[list(aboveCenter)])<np.array(centers[list(aboveCenter)])).replace(False,np.NaN).dropna().index]     
        
        oppositefilter = oppositefilter.iloc[pd.DataFrame(np.array(oppositefilter[list(belowCenter)])>np.array(centers[list(belowCenter)])).replace(False,np.NaN).dropna().index]    
        #print("filtered:",filtered.index)
        #print("opposite:",oppositefilter.index)
        together = np.concatenate([filtered.index,oppositefilter.index])
        #print("together",together)
        neither = [i for i, item in enumerate(list(data.index)) if item not in together]
        #print("neither",neither)
        neitherfilter = data.copy()
        neitherfilter = neitherfilter.iloc[neither]
        #print(neitherfilter.index)
        
        '''
        A=notfiltered
        B=filtered
        H0: A>=B
        H1: B>A
        '''
        
        quantiles_7num = [.02,.09,.25,.5,.75,.91,.98]
        
        q_s_a = data[y].quantile(quantiles_7num)
        q_s_i = filtered[y].quantile(quantiles_7num)
        q_s_o = oppositefilter[y].quantile(quantiles_7num)
        q_s_n = neitherfilter[y].quantile(quantiles_7num)
       
        data1 = pd.DataFrame(data[[y]]).assign(Location=1)[['Location',y]]
        data2 = pd.DataFrame(filtered[y]).assign(Location=2)[['Location',y]]
        data3 = pd.DataFrame(oppositefilter[y]).assign(Location=3)[['Location',y]]
        data4 = pd.DataFrame(neitherfilter[y]).assign(Location=4)[['Location',y]]
        #print(data2.index)
        #print(data3.index)
        #print(data4.index)
        #print(data4)
        
        #if(len(together)>0):
        data5 = pd.concat([data2,data3,data4],axis=0)#[['Location']]

        data5 = data5.sort_values(kind="quicksort", by=y, ascending=True)[['Location']]
        #data.loc[data5.index]['cluster']=data5['Location']

        #print(data5.index)
        #print(data.index)
        data['cluster'] = data5['Location']

        state_vector = []
        #us_state_to_abbrev[[item for item in df.index]]
        for i in data.index:
            state_vector.append(us_state_to_abbrev.get(i, 0))

        #data['States']=state_vector
        data['States']=data.index
        #print(data5)
        #data['group']=data5
        #print(data)

        with localconverter(ro.default_converter + pandas2ri.converter):
            r_from_pd_df = ro.conversion.py2rpy(data)
            d=(r_f(r_from_pd_df))

        image = Image.open('C:/Users/User/Documents/wiki/wiki/dev/R/clusterMap.png')
        #image.show()
        display(image)

        cdf = pd.concat([data1, data2, data3, data4])    
        mdf = pd.melt(cdf, id_vars=['Location'], var_name=['Group'])
        #print(mdf.head())

        ax3 = sns.boxplot(x="Location", y="value", hue="Group", data=mdf)    
        plt.show()
        
        #fig = mpf.figure(figsize=(10, 7), style=s)
        fig = plt.figure(figsize=plt.figaspect(2.))
        fig.suptitle('boxplots')
        #fig = Figure()
        #FigureCanvas(fig) # not needed in mpl >= 3.1
        ax1 = fig.add_subplot(6,2,1)
        ax2 = fig.add_subplot(6,2,2)
        ax3 = fig.add_subplot(6,2,3)
        ax4 = fig.add_subplot(6,2,4)
        ax5 = fig.add_subplot(6,2,5)
        ax6 = fig.add_subplot(6,2,6)
        
        print("all mapped against either group")
        print("all mapped against neither group")
        print("7 number ogive all mapped with in and out group")
        print("7 number ogive all mapped against neither group")
        ax1.hist(data[y])
        ax1.hist(filtered[y])
        ax1.hist(oppositefilter[y])
        ax2.hist(data[y])
        ax2.hist(neitherfilter[y])
        ax3.plot(data[y])
        ax3.plot(filtered[y])
        ax3.plot(oppositefilter[y])
        #ax3.xticks(rotation = 45,size=8)
        ax4.plot(q_s_a)
        ax4.plot(q_s_i)
        ax4.plot(q_s_o)
        ax5.plot(data[y])
        ax5.plot(neitherfilter[y])
        #ax5.xticks(rotation = 45,size=8)
        ax6.plot(q_s_a)
        ax6.plot(q_s_n)
        plt.show()        
        
        print("in group")
        sp = (st.ttest_ind(filtered[y],data[y],equal_var=False))
        print(sp.pvalue/2)
        #print(st.ttest_1samp(a=filtered[y],popmean=np.mean(data[y])))
        print("f/variance test:",f_test(filtered[y],data[y]))
        #if(sp.pvalue/2<.05):
        #print("reject null hypothesis, B(filter)>A")
        #df_ = len(filtered[y]) + len(data[y]) - 2
        display(filtered.describe())
        display(filtered)

        print("out group")
        sp = (st.ttest_ind(oppositefilter[y],data[y],equal_var=False))
        print(sp.pvalue/2)
        #print(st.ttest_1samp(a=oppositefilter[y],popmean=np.mean(data[y])))
        print("f/variance test:",f_test(oppositefilter[y],data[y]))
        #if(sp.pvalue/2<.05):
            #print("reject null hypothesis, B(filter)>A")
        #df_ = len(filtered[y]) + len(data[y]) - 2
        display(oppositefilter.describe())
        display(oppositefilter)

        #return([filtered,oppositefilter])
        print("neither group")
        sp = (st.ttest_ind(neitherfilter[y],data[y],equal_var=False))
        print(sp.pvalue/2)
        #print(st.ttest_1samp(a=neitherfilter[y],popmean=np.mean(data[y])))
        print("f/variance test:",f_test(neitherfilter[y],data[y]))
        display(neitherfilter.describe())
        display(neitherfilter)

    aboveCenter = widgets.SelectMultiple(
        options=df_.columns,
        #options=df[list(x)].columns,
        description='AboveCenter',
        disabled=False
    )
    
    belowCenter = widgets.SelectMultiple(
        options=df_.columns,
        #options=df[list(x)].columns,
        description='BelowCenter',
        disabled=False
    )
    
    center=widgets.Checkbox(
        description='Center: [Checked: Mean, Unchecked: Median]',
        value=False,
        disabled=False,
        indent=False)

    interact(print_df,aboveCenter=aboveCenter,belowCenter=belowCenter,center=center)
    
def derive_xnames(y):
    x_ = widgets.SelectMultiple(
        options=df.columns[~df.columns.isin([y])],
        value=list(df.columns[~df.columns.isin([y])]),
        disabled=False
    )
    
    interact(filter_df,x=x_,y=y)

In [12]:
#abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [29]:

y=widgets.Select(options=df_.columns,disabled=False)

interact(derive_xnames,y=y)


interactive(children=(Select(description='y', options=('Poverty', 'Infant Mort', 'White', 'Crime', 'Doctors', …

<function __main__.derive_xnames(y)>